In [2]:
from openai import OpenAI
import openai
import os
import ast
import re
import json
import markdown
from bs4 import BeautifulSoup
import pickle

import requests
from xml.etree import ElementTree
import ast

from collections import defaultdict
from difflib import SequenceMatcher
import numpy as np
from scipy.cluster.hierarchy import fcluster
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from linkml_runtime.loaders.yaml_loader import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView

In [4]:
openai.api_key = ""

In [5]:
# Initialize the OpenAI client
client = OpenAI(api_key=openai.api_key)#, base_url="https://api.cborg.lbl.gov")

In [6]:
# Define the function to query the OpenAI API
def query_openai(model, role, prompt):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": role, "content": prompt}
            ]
        )
        return response#.choices[0].message['content'] if response.choices else None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [7]:
#metadata-elements-comparison.txt


In [7]:
schema_path = "../src/data_sheets_schema/schema/data_sheets_schema.yaml"
# Load schema
#yaml_loader = YAMLLoader()
schema_view = SchemaView(schema_path)

print(f"Schema name: {schema_view.schema.name}")
classes = schema_view.all_classes()
print(f"Classes in the schema: {list(classes.keys())}")

INFO:root:Importing standards_organization_schema as standards_organization_schema from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=../src/data_sheets_schema/schema
INFO:root:Importing standards_schema as standards_schema from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=../src/data_sheets_schema/schema
INFO:root:Importing linkml:types as /Users/marcin/Documents/VIMSS/ontology/bridge2ai/data-sheets-schema/venv/lib/python3.10/site-packages/linkml_runtime/linkml_model/model/schema/types from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=None


Schema name: data-sheets-schema
Classes in the schema: ['NamedThing', 'AnatomicalEntity', 'Organization', 'OrganizationContainer', 'Information', 'FormatDialect', 'Person', 'DatasetProperty', 'DatasetCollection', 'Dataset', 'DataSubset', 'Software', 'Purpose', 'Task', 'AddressingGap', 'Creator', 'FundingMechanism', 'Grantor', 'Grant', 'Instance', 'SamplingStrategy', 'MissingInfo', 'Relationships', 'Splits', 'DataAnomaly', 'ExternalResource', 'Confidentiality', 'ContentWarning', 'Subpopulation', 'Deidentification', 'SensitiveElement', 'InstanceAcquisition', 'CollectionMechanism', 'DataCollector', 'CollectionTimeframe', 'EthicalReview', 'DirectCollection', 'CollectionNotification', 'CollectionConsent', 'ConsentRevocation', 'DataProtectionImpact', 'PreprocessingStrategy', 'CleaningStrategy', 'LabelingStrategy', 'RawData', 'ExistingUse', 'UseRepository', 'OtherTask', 'FutureUseImpact', 'DiscouragedUse', 'ThirdPartySharing', 'DistributionFormat', 'DistributionDate', 'LicenseAndUseTerms', 'I

In [8]:
schema_view

SchemaView(schema=SchemaDefinition({
  'name': 'data-sheets-schema',
  'description': 'A LinkML schema for Datasheets for Datasets.',
  'title': 'data-sheets-schema',
  'see_also': ['https://bridge2ai.github.io/data-sheets-schema'],
  'id': 'https://w3id.org/bridge2ai/data-sheets-schema',
  'imports': ['linkml:types', 'standards_schema', 'standards_organization_schema'],
  'license': 'MIT',
  'prefixes': {'biolink': Prefix({'prefix_prefix': 'biolink', 'prefix_reference': 'https://w3id.org/biolink/vocab/'}),
    'csvw': Prefix({'prefix_prefix': 'csvw', 'prefix_reference': 'http://www.w3.org/ns/csvw#'}),
    'data_sheets_schema': Prefix({
      'prefix_prefix': 'data_sheets_schema',
      'prefix_reference': 'https://w3id.org/bridge2ai/data-sheets-schema/'
    }),
    'datasets': Prefix({'prefix_prefix': 'datasets', 'prefix_reference': 'https://w3id.org/linkml/report'}),
    'dcat': Prefix({'prefix_prefix': 'dcat', 'prefix_reference': 'http://www.w3.org/ns/dcat#'}),
    'example': Prefix

In [9]:
# Get all classes
classes = schema_view.all_classes()
for class_name, class_def in classes.items():
    print(f"Class: {class_name}, Definition: {class_def}")

# Get all slots
slots = schema_view.all_slots()
for slot_name, slot_def in slots.items():
    print(f"Slot: {slot_name}, Definition: {slot_def}")

# Get all enums
enums = schema_view.all_enums()
for enum_name, enum_def in enums.items():
    print(f"Enum: {enum_name}, Definition: {enum_def}")


Class: NamedThing, Definition: ClassDefinition({
  'name': 'NamedThing',
  'description': 'A generic grouping for any identifiable entity',
  'from_schema': 'https://w3id.org/bridge2ai/standards-schema',
  'slots': ['id', 'category', 'name', 'description', 'subclass_of', 'related_to',
    'contributor_name', 'contributor_github_name', 'contributor_orcid',
    'contribution_date'],
  'class_uri': 'schema:Thing'
})
Class: AnatomicalEntity, Definition: ClassDefinition({
  'name': 'AnatomicalEntity',
  'id_prefixes': ['uberon'],
  'description': 'A subcellular location, cell type or gross anatomical part',
  'from_schema': 'https://w3id.org/bridge2ai/standards-schema',
  'exact_mappings': ['uberon:0001062', 'wikidata:Q4936952'],
  'narrow_mappings': ['ncit:C12219'],
  'is_a': 'NamedThing'
})
Class: Organization, Definition: ClassDefinition({
  'name': 'Organization',
  'description': ('Represents a group or organization related to or responsible for one or more '
     'Bridge2AI standards.

In [10]:
class_details = {}

for class_name, class_def in schema_view.all_classes().items():
    description = class_def.description if class_def.description else 'No description available'
    subsets = class_def.in_subset if class_def.in_subset else []
    attributes = {}
    for attr_name, attr_def in class_def.attributes.items():
        attributes[attr_name] = attr_def.description if attr_def.description else 'No description available'
    
    class_details[class_name] = {
        'subset': subsets,
        'attributes': attributes,
        'description': description
    }

# Print the resulting dictionary for verification
for class_name, details in class_details.items():
    print(f"Class: {class_name}")
    print(f"  Subsets: {details['subset']}")
    print(f"  Description: {details['description']}")
    print(f"  Attributes:")
    for attr_name, attr_desc in details['attributes'].items():
        print(f"    - {attr_name}: {attr_desc}")
    print()

Class: NamedThing
  Subsets: []
  Description: A generic grouping for any identifiable entity
  Attributes:

Class: AnatomicalEntity
  Subsets: []
  Description: A subcellular location, cell type or gross anatomical part
  Attributes:

Class: Organization
  Subsets: []
  Description: Represents a group or organization related to or responsible for one or more Bridge2AI standards.
  Attributes:

Class: OrganizationContainer
  Subsets: []
  Description: A container for Organizations.
  Attributes:

Class: Information
  Subsets: []
  Description: Grouping for datasets and data files
  Attributes:

Class: FormatDialect
  Subsets: []
  Description: Additional format information for a file
  Attributes:
    - comment_prefix: No description available
    - delimiter: No description available
    - double_quote: No description available
    - header: No description available
    - quote_char: No description available

Class: Person
  Subsets: []
  Description: An individual human being.
  Attr

In [11]:
# Prepare the class details as a formatted string for the prompt
class_details_str = "\n".join(
    f"Class name: {class_name}\n"
    f"  Subsets: {', '.join(details['subset'])}\n"
    f"  Description: {details['description']}\n"
    f"  Attributes:\n" +
    "".join(f"    {attr_name}: {attr_desc}\n" for attr_name, attr_desc in details['attributes'].items())
    for class_name, details in class_details.items()
)


In [16]:
import pandas as pd

# Try reading the file with 'latin-1' encoding
try:
    healthsheet_structure = pd.read_csv("./metadata-elements__healtsheet_breakdown.txt", sep="\t", header=0, index_col=0, encoding='latin-1')
    print(healthsheet_structure.shape)
except UnicodeDecodeError:
    print("Failed to read file with 'latin-1' encoding.")
    # If the above fails, try reading the file with 'windows-1252' encoding
    try:
        healthsheet_structure = pd.read_csv("./metadata-elements__healtsheet_breakdown.txt", sep="\t", header=0, index_col=0, encoding='windows-1252')
        print(healthsheet_structure.shape)
    except UnicodeDecodeError:
        print("Failed to read file with 'windows-1252' encoding.")


healthsheet_structure

(272, 7)


,Question or field,Metadata reference code,Metadata description,Is controlled value,Metadata code,Comment (Anna),Is in datasheet
Reference code,,,,,,,
General_information.1,Provide a 2 sentence summary of this dataset.,General_information.1,Brief description of the dataset (2 sentences),False,dataset_description,NaN,FALSE
General_information.2,"Has the dataset been audited before? If yes, b...",General_information.2.1,Indication of whether the dataset has been aud...,False,has_audit,NaN,FALSE
NaN,NaN,General_information.2.2,"If the dataset has been audited, indication wh...",False,who_audit,NaN,FALSE
NaN,NaN,General_information.2.3,"If the dataset has been audited, indication of...",False,results_audit,NaN,FALSE
Dataset_versioning.1,Does the dataset get released as static versio...,Dataset_versioning.1.1,Indication of whether the dataset is static or...,False,is_static_or_dynamic,NaN,FALSE
...,...,...,...,...,...,...,...
NaN,NaN,Maintenance.7.5,"If applicable, description of why external con...",False,why_no_validation_external_contribution,NaN,TRUE
NaN,NaN,Maintenance.7.6,"If applicable, indication of whether there is ...",False,has_communication_external_contribution,NaN,TRUE
NaN,NaN,Maintenance.7.7,"If applicable, description of how external con...",False,description_communication_external_contribution,NaN,TRUE


In [13]:
# Read the Markdown file content
with open("./healthsheet.md", 'r', encoding='utf-8') as file:
    markdown_content = file.read()

# Convert the Markdown content to HTML
html_content = markdown.markdown(markdown_content)

print(html_content)

<h1>Healthsheet for AI-READI Dataset</h1>
<p>This healthsheet provides information about the AI-READI dataset in line with a previously established template (Rostamzadeh et al., https://arxiv.org/abs/2202.13028) aimed at facilitating consistent and transparent documentation regarding the creation, use, and maintenance of healthcare datasets for machine learning.</p>
<h2>General information</h2>
<ol>
<li>Provide a 2 sentence summary of this dataset.</li>
</ol>
<p>The Artificial Intelligence Ready and Equitable Atlas for Diabetes Insights (AI-READI) is a dataset consisting of data collected from individuals with and without "Type 2 Diabetes Mellitus (T2DM)" and harmonized across 3 data collection sites. The composition of the dataset was designed with future studies using AI/Machine Learning in mind. This included recruitment sampling procedures aimed at achieving approximately equal distribution of participants across sex, race, and diabetes severity, as well as the design of a data acq

In [14]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'lxml')

# Find all <li> tags and print them
list_items = soup.find_all('li')
for item in list_items:
    print("* "+item.get_text())

* Provide a 2 sentence summary of this dataset.
* Has the dataset been audited before? If yes, by whom and what are the results?
* Does the dataset get released as static versions or is it dynamically updated?
* Is this datasheet created for the original version of the dataset? If not, which version of the dataset is this datasheet for?
* Are there any datasheets created for any versions of this dataset?
* Does the current version/subversion of the dataset come with predefined task(s), labels, and recommended data splits (e.g., for training, development/validation, testing)? If yes, please provide a high-level description of the introduced tasks, data splits, and labeling, and explain the rationale behind them. Please provide the related links and references. If not, is there any resource (website, portal, etc.) to keep track of all defined tasks and/or associated label definitions? (please note that more detailed questions w.r.t labeling is provided in further sections)
* If the datas

In [ ]:
#Partial matches should only be reported if there is no exact or close match.


In [67]:
llm_prompt_template = """
You are a biomedical semantic curator and engineer working on comparing, aligning, and merging two complex biomedical schemas for health data.

You are given a Healthsheet Metadata Code along with some associated fields for context. 
You are also given a dictionary of class details from another schema. 
Your tasks are as follows:

1. **Identify Relevant Class Attributes**: Based on the other schema consisting of Class, Subsets, Description, and Attributes, find the most relevant Class Attribute for this Metadata Code.
   - If multiple class attributes are found to match, list all. This may involve different Classes and multiple Attributes from the same Class.
   - If no reasonable match is found, report "No match" in the 'Closeness' field and leave the others empty.
3. **Describe Match Type**: Use the Closeness field to describe the match type: "exact", "close", or "partial".

Make sure that the entire schema is considered and that all exact and close matches are reported. Double check to make sure this is the case.

Format the output as in the example below, ensuring that matches to different Class Attributes for this metadata code will have separate output sections in this output format.
Do not add any other text or formatting to the output and make sure to adhere to this output format including naming, structuring, and ordering of fields.

Example exact matches:
**Healthsheet entry:** "dataset_description"\n  - **Closeness:** "exact"\n  - **Class:** "Dataset"\n  - **Attribute:** "description"
*Healthsheet entry:** "is_static_or_dynamic"\n  - **Closeness:** "exact"\n  - **Class:** "Dataset"\n  - **Attribute:** "updates"

Example close matches:
**Healthsheet entry:** "dataset_description"\n  - **Closeness:** "close"\n  - **Class:** "Purpose"\n  - **Attribute:** "response"
**Healthsheet entry:** "has_audit"\n  - **Closeness:** "close"\n  - **Class:** "Dataset"\n  - **Attribute:** "ethical_reviews"'

Example partial matches:
**Healthsheet entry:** "has_audit"\n  - **Closeness:** "partial"\n  - **Class:** "Dataset"\n  - **Attribute:** "data_protection_impacts"
**Healthsheet entry:** "who_audit"\n  - **Closeness:** "partial"\n  - **Class:** "ExistingUse"\n  - **Attribute:** "description"
**Healthsheet entry:** "who_audit"\n  - **Closeness:** "partial"\n  - **Class:** "Dataset"\n  - **Attribute:** "data_collectors"

Example no matches entry:
**Healthsheet entry:** "destruction_date"\n  - **Closeness:** "No match"\n  - **Class:** ""\n  - **Attribute:** ""


Output format with variable to complete by LLM:

**Healthsheet entry:** "<metadata code>"
  - **Closeness:** "<closeness1>"
  - **Class:** "<class_name1>"
  - **Attribute:** "<attribute name1>"
**Healthsheet entry:** "<metadata code>"
  - **Closeness:** "<closeness2>"
  - **Class:** "<class_name1>"
  - **Attribute:** "<attribute name2>"
**Healthsheet entry:** "<metadata code>"
  - **Closeness:** "<closeness3>"
  - **Class:** "<class_name2>"
  - **Attribute:** "<attribute name3>"
---

Here is the Healthsheet data row for the specific metadata code and its other associated fields for context:

- Metadata Code: {metadata_code}
- Reference Code: {reference_code}
- Question: {question}
- Metadata Description: {metadata_description}


Here are the full class details from the other schema:

{class_details}

"""

In [48]:
healthsheet_structure.index

Index(['General_information.1', 'General_information.2',
                           nan,                     nan,
        'Dataset_versioning.1',                     nan,
                           nan,  'Dataset_versioning.2',
                           nan,  'Dataset_versioning.3',
       ...
                           nan,         'Maintenance.7',
                           nan,                     nan,
                           nan,                     nan,
                           nan,                     nan,
               'Maintenance.8',                     nan],
      dtype='object', name='Reference code', length=272)

In [49]:
healthsheet_structure.reset_index(inplace=True) 
healthsheet_structure

,Reference code,Question or field,Metadata reference code,Metadata description,Is controlled value,Metadata code,Comment (Anna),Is in datasheet
0,General_information.1,Provide a 2 sentence summary of this dataset.,General_information.1,Brief description of the dataset (2 sentences),False,dataset_description,NaN,FALSE
1,General_information.2,"Has the dataset been audited before? If yes, b...",General_information.2.1,Indication of whether the dataset has been aud...,False,has_audit,NaN,FALSE
2,NaN,NaN,General_information.2.2,"If the dataset has been audited, indication wh...",False,who_audit,NaN,FALSE
3,NaN,NaN,General_information.2.3,"If the dataset has been audited, indication of...",False,results_audit,NaN,FALSE
4,Dataset_versioning.1,Does the dataset get released as static versio...,Dataset_versioning.1.1,Indication of whether the dataset is static or...,False,is_static_or_dynamic,NaN,FALSE
...,...,...,...,...,...,...,...,...
267,NaN,NaN,Maintenance.7.5,"If applicable, description of why external con...",False,why_no_validation_external_contribution,NaN,TRUE
268,NaN,NaN,Maintenance.7.6,"If applicable, indication of whether there is ...",False,has_communication_external_contribution,NaN,TRUE
269,NaN,NaN,Maintenance.7.7,"If applicable, description of how external con...",False,description_communication_external_contribution,NaN,TRUE
270,Maintenance.8,Any other comments?,Maintenance.8,Additional comments regarding dataset maintenance,False,additional_comments_dataset_maintenance,NaN,TRUE


In [56]:
llm_results = []

# Function to strip HTML/Markdown tags
def strip_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Loop over Healthsheets structure entries, fill out the prompt template, and submit to the LLM
for index, row in healthsheet_structure.iterrows():
    reference_code = row['Metadata reference code']
    question = row['Question or field']
    metadata_description = row['Metadata description']
    metadata_code = row['Metadata code']

    if pd.isna(reference_code) or pd.isna(question):
        # Ensure index is an integer
        index = int(index)
        # Search for the first earlier row with non-NaN values for Reference Code and Question
        for prev_index in range(index - 1, -1, -1):
            prev_row = healthsheet_structure.iloc[prev_index]
            if pd.notna(prev_row['Metadata reference code']) and pd.notna(prev_row['Question or field']):
                reference_code = prev_row['Metadata reference code']
                question = prev_row['Question or field']
                break
    
    prompt = llm_prompt_template.format(
        reference_code=reference_code,
        question=question,
        metadata_description=metadata_description,
        metadata_code=metadata_code,
        class_details=class_details_str
    )
    
    # Call to LLM (Replace with actual LLM call)
    result = query_openai("gpt-4o", "user", prompt)  # Uncomment this for actual LLM call
    # result = f"Simulated result for prompt: {prompt}"  # Simulated result for demonstration

    # print(prompt)  # For debugging
    print(result)
    llm_results.append(result)
    
# Output the results for verification
for result in llm_results:
    print(result)
    print("------")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-A6XTdC7ctfmJQmWNQ33AOIM3G7ho1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Healthsheet entry:** "dataset_description"\n  - **Closeness:** "exact"\n  - **Class:** "Dataset"\n  - **Attribute:** "description"\n**Healthsheet entry:** "dataset_description"\n  - **Closeness:** "partial"\n  - **Class:** "ExistingUse"\n  - **Attribute:** "description"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726122017, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_25624ae3a5', usage=CompletionUsage(completion_tokens=71, prompt_tokens=5428, total_tokens=5499))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-A6XTfBVe0u76opYyPxK9iG8SfXkSp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Healthsheet entry:** "has_audit"\n  - **Closeness:** "close"\n  - **Class:** "Dataset"\n  - **Attribute:** "data_protection_impacts"\n**Healthsheet entry:** "has_audit"\n  - **Closeness:** "close"\n  - **Class:** "Dataset"\n  - **Attribute:** "ethical_reviews"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726122019, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_25624ae3a5', usage=CompletionUsage(completion_tokens=77, prompt_tokens=5442, total_tokens=5519))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

ChatCompletion(id='chatcmpl-A6XbC5cpfsQOH8Pe7h6gauqW6fhaU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Healthsheet entry:** "description_communication_external_contribution"\n  - **Closeness:** "exact"\n  - **Class:** "ExtensionMechanism"\n  - **Attribute:** "description"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726122486, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_25624ae3a5', usage=CompletionUsage(completion_tokens=41, prompt_tokens=5512, total_tokens=5553))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-A6XbDuDb8GlzMql1c39c3c5ta2mKR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Healthsheet entry:** "additional_comments_dataset_maintenance"\n  - **Closeness:** "exact"\n  - **Class:** "UpdatePlan"\n  - **Attribute:** "description"\n**Healthsheet entry:** "additional_comments_dataset_maintenance"\n  - **Closeness:** "partial"\n  - **Class:** "Maintainer"\n  - **Attribute:** "description"\n**Healthsheet entry:** "additional_comments_dataset_maintenance"\n  - **Closeness:** "partial"\n  - **Class:** "Erratum"\n  - **Attribute:** "description"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726122487, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_25624ae3a5', usage=CompletionUsage(completion_tokens=117, prompt_tokens=5421, total_tokens=5538))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-A6XbFuGUacerKq28muCEW2njCiA3m', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Healthsheet entry:** "nan"\n  - **Closeness:** "partial"\n  - **Class:** "FormatDialect"\n  - **Attribute:** "comment_prefix"\n**Healthsheet entry:** "nan"\n  - **Closeness:** "partial"\n  - **Class:** "Dataset"\n  - **Attribute:** "errata"\n', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726122489, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_25624ae3a5', usage=CompletionUsage(completion_tokens=71, prompt_tokens=5413, total_tokens=5484))
ChatCompletion(id='chatcmpl-A6XTdC7ctfmJQmWNQ33AOIM3G7ho1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Healthsheet entry:** "dataset_description"\n  - **Closeness:** "exact"\n  - **Class:** "Dataset"\n  - **Attribute:** "description"\n**Healthsheet

In [77]:
# Function to safely extract data between two markers
def extract_between(content, start=None, end=None):
    try:
        if start and end:
            return content.split(start, 1)[1].split(end, 1)[0].strip()
        elif start:
            return content.split(start, 1)[1].strip()
        elif end:
            return content.split(end, 1)[0].strip()
    except IndexError:
        return "Not Found"

# Parse the LLM responses and extract the required data
parsed_data = []

for response in llm_results:
    try:
        response_content = response.choices[0].message.content
        # Split the response content into individual Healthsheet entries
        entries = response_content.split('**Healthsheet entry:** "')[1:]  # Split and remove the first empty split section
        
        for entry in entries:
            # Process individual Healthsheet entries
            entry_text = extract_between(entry, "", '"')
            closeness = extract_between(entry, '- **Closeness:** "', '"')
            class_name = extract_between(entry, '- **Class:** "', '"')
            attribute = extract_between(entry, '- **Attribute:** "', '"')
            
            parsed_data.append({
                "Healthsheet entry": entry_text,
                "Closeness": closeness,
                "Class": class_name,
                "Attribute": attribute
            })

    except Exception as e:
        print(f"An error occurred while parsing response: {response.id} - {e}")

# Create a DataFrame with the extracted data
df = pd.DataFrame(parsed_data, columns=["Healthsheet entry", "Closeness", "Class", "Attribute"])

# Determine if each Healthsheet entry has an exact match or only partial matches
grouped = df.groupby('Healthsheet entry')['Closeness'].apply(list).reset_index()

# Create HasExact and OnlyPartial columns
grouped['HasExact'] = grouped['Closeness'].apply(lambda x: 'yes' if 'exact' in x else 'no')
grouped['OnlyPartial'] = grouped['Closeness'].apply(lambda x: 'yes' if all(c == 'partial' for c in x) else 'no')

# Merge the new columns back into the original DataFrame
df = df.merge(grouped[['Healthsheet entry', 'HasExact', 'OnlyPartial']], on='Healthsheet entry', how='left')

# Save the DataFrame to a TSV file
df.to_csv('healthsheet_into_datasheet_codes_llm_results_v4.tsv', sep='\t', index=False)

df

,Healthsheet entry,Closeness,Class,Attribute,HasExact,OnlyPartial
0,dataset_description,exact,Dataset,description,yes,no
1,dataset_description,partial,ExistingUse,description,yes,no
2,has_audit,close,Dataset,data_protection_impacts,no,no
3,has_audit,close,Dataset,ethical_reviews,no,no
4,who_audit,partial,Dataset,data_collectors,no,yes
...,...,...,...,...,...,...
662,additional_comments_dataset_maintenance,exact,UpdatePlan,description,yes,no
663,additional_comments_dataset_maintenance,partial,Maintainer,description,yes,no
664,additional_comments_dataset_maintenance,partial,Erratum,description,yes,no
665,nan,partial,FormatDialect,comment_prefix,no,yes


In [60]:
# Save the object to a file
with open('data_vs_health_sheet_codes_alignment_raw_v4.pkl', 'wb') as file:
    pickle.dump(llm_results, file)

In [ ]:
#with open('data_vs_health_sheet_code_alignment_raw_v4.pkl', 'rb') as file:
#    llm_results = pickle.load(file)

In [64]:
df

,Healthsheet entry,Closeness,Class,Attribute
0,dataset_description,exact,Dataset,description
1,dataset_description,partial,ExistingUse,description
2,has_audit,close,Dataset,data_protection_impacts
3,has_audit,close,Dataset,ethical_reviews
4,who_audit,partial,Dataset,data_collectors
...,...,...,...,...
662,additional_comments_dataset_maintenance,exact,UpdatePlan,description
663,additional_comments_dataset_maintenance,partial,Maintainer,description
664,additional_comments_dataset_maintenance,partial,Erratum,description
665,nan,partial,FormatDialect,comment_prefix


In [65]:
df[df['Closeness'] == 'exact']

,Healthsheet entry,Closeness,Class,Attribute
0,dataset_description,exact,Dataset,description
9,is_static_or_dynamic,exact,Dataset,updates
11,number_of_existing_versions,exact,Dataset,instances
12,frequency_update,exact,Dataset,updates
13,datasheet_is_for_original_version,exact,Dataset,version_access
...,...,...,...,...
658,why_no_validation_external_contribution,exact,ExtensionMechanism,description
659,has_communication_external_contribution,exact,Dataset,extension_mechanism
660,has_communication_external_contribution,exact,ExtensionMechanism,description
661,description_communication_external_contribution,exact,ExtensionMechanism,description


In [73]:
# Group by 'Healthsheet entry' and check for the presence of 'exact' closeness
unique_entries_with_exact = df.groupby('Healthsheet entry').apply(lambda x: 'exact' in x['Closeness'].values)
exact_entries = unique_entries_with_exact[unique_entries_with_exact].index

# Count the number of unique Healthsheet entries with an exact match
count_exact_entries = len(exact_entries)
count_exact_entries

/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_72910/3343744260.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_entries_with_exact = df.groupby('Healthsheet entry').apply(lambda x: 'exact' in x['Closeness'].values)


217

In [74]:
# Group by 'Healthsheet entry' and check for the presence of 'exact' closeness
unique_entries = df.groupby('Healthsheet entry').apply(lambda x: 'exact' not in x['Closeness'].values)
non_exact_entries = unique_entries[unique_entries].index

# Count the number of unique Healthsheet entries without an exact match
count_non_exact_entries = len(non_exact_entries)

count_non_exact_entries

/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_72910/1477071054.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_entries = df.groupby('Healthsheet entry').apply(lambda x: 'exact' not in x['Closeness'].values)


48

In [ ]:
271 vs 217 + 48 = 265

In [49]:
healthsheet_to_d4d_mapping_df[healthsheet_to_d4d_mapping_df['Class'] == 'Not Found']

,Healthsheet entry,Closeness,Class,Subset,Attributes
40,Not Found,Not Found,Not Found,Not Found,Not Found
72,Not Found,Not Found,Not Found,Not Found,Not Found
124,Not Found,Not Found,Not Found,Not Found,Not Found
150,Not Found,Not Found,Not Found,Not Found,Not Found
213,Not Found,Not Found,Not Found,Not Found,Not Found
222,Not Found,Not Found,Not Found,Not Found,Not Found
225,Not Found,Not Found,Not Found,Not Found,Not Found
229,Not Found,Not Found,Not Found,Not Found,Not Found
